# Upgrading from MathProgBase.jl to MathOptInterface.jl

+ MathProgBase.jl has been [depreciated](https://github.com/JuliaOpt/MathProgBase.jl/pull/231) since Jan 2019. It is therefore not officially supported by julia v1.2 and above. We should use [MathOptInterface.jl](https://github.com/jump-dev/MathOptInterface.jl) instead. 
+ Other packages such as BinaryProvider.jl and Ipopt.jl needs to also be updated to the latest version or depreciated and use the latest one supported by the Julia community.
+ To ensure a smooth update, checkout the [official documentation](https://jump.dev/MathOptInterface.jl/dev/tutorials/mathprogbase/) as well as examples by [WiSER.jl](https://github.com/OpenMendel/WiSER.jl/blob/master/src/fit.jl) and [OrdinalMultinomialModels.jl](https://github.com/OpenMendel/OrdinalMultinomialModels.jl/blob/master/src/ordmnfit.jl)

In [1]:
using Revise
using Ipopt
using QuasiCopula
using Random
using GLM
using LinearAlgebra
using Statistics
using StatsBase
using MathOptInterface
using RCall

ENV["COLUMNS"] = 240
BLAS.set_num_threads(1)
Threads.nthreads()

Precompiling QuasiCopula
  ✓ QuasiCopula
  1 dependency successfully precompiled in 5 seconds. 151 already precompiled.


1

In [16]:
k = 0 # number of causal SNPs

qc_model, G, βtrue, θtrue, γtrue, τtrue = simulate_longitudinal_traits(
    n = 10000, # sample size
    d_min = 5, # min number of observations per sample
    d_max = 5, # max number of observations per sample
    p = 15, # number of fixed effects, including intercept
    m = 2, # number of variance components
    q = 1000, # number of SNPs
    k = k, # number of causal SNPs
    seed = 2024,
    y_distribution = NegativeBinomial,
    τtrue = 0.5,
    T = Float64,
    maf = 0.3,
    causal_snp_β = 0.2
)

@show qc_model;

@time optm = QuasiCopula.fit!(qc_model);

initializing β using GLM.jl
gcm.β = [1.071141333938132, -0.4820165636340641, 0.48187754351752365, 0.48479332423135835, 0.49028177661345046, -0.4862122196580561, 0.48011767932560134, -0.4909547903170821, 0.48343181680091807, -0.48629241376979127, -0.4819929966888139, -0.48739697346794014, 0.4847745221649089, -0.48215713945837935, -0.4820357318329954]
initializing variance components using MM-Algorithm
gcm.θ = [1.0592722715027482e-6, 6.122833003834026e-8]
initializing r using Newton update
qc_model = Quasi-Copula Variance Component Model
  * base distribution: NegativeBinomial
  * link function: LogLink
  * number of clusters: 10000
  * cluster size min, max: 5, 5
  * number of variance components: 2
  * number of fixed effects: 15

initializing β using GLM.jl
gcm.β = [1.071141333938132, -0.4820165636340641, 0.48187754351752365, 0.48479332423135835, 0.49028177661345046, -0.4862122196580561, 0.48011767932560134, -0.4909547903170821, 0.48343181680091807, -0.48629241376979127, -0.4819929966

In [17]:
@show βtrue
@show qc_model.β
@show qc_model.∇β

@show θtrue
@show qc_model.θ
@show qc_model.∇θ;

@show τtrue
@show qc_model.τ
@show qc_model.∇τ;

βtrue = [1.0, -0.5, 0.5, 0.5, 0.5, -0.5, 0.5, -0.5, 0.5, -0.5, -0.5, -0.5, 0.5, -0.5, -0.5]
qc_model.β = [1.004605601708639, -0.49848912400485396, 0.49751094935089923, 0.49863406339611727, 0.5005358158505578, -0.49937733166336645, 0.49609926123970455, -0.5030694132784159, 0.49790902027737355, -0.4987235407180179, -0.49798418594541544, -0.5024552471390116, 0.499394490138676, -0.4974251284604497, -0.4978962329135943]
qc_model.∇β = [2.5378592177815293, -0.3878816537591483, 0.48456340051407076, 0.6087883728777483, 0.18267478283986938, -0.47801167343143813, 0.6235818600406728, -0.14411185012481154, 0.46035500849368904, -0.3316749927358775, -0.024800958980559007, -0.532825513048496, 0.32461402813598816, -0.4421443779961307, -0.5799522588043043]
θtrue = [0.1, 0.1]
qc_model.θ = [0.10011285253205283, 0.08230766496943064]
qc_model.∇θ = [6.588716663682135, 7.916188025443262]
τtrue = 0.5
qc_model.τ = [1.0]
qc_model.∇τ = [0.0]


In [18]:
qc_model.r # rtrue = 10 (negative binomial only)

1-element Vector{Float64}:
 10.026348269269533

# AR

In [1]:
using Revise
using Ipopt
using QuasiCopula
using Random
using GLM
using LinearAlgebra
using Statistics
using StatsBase
using MathOptInterface
using RCall

Precompiling QuasiCopula
  ✓ QuasiCopula
  1 dependency successfully precompiled in 4 seconds. 151 already precompiled.


In [15]:
# get data
R"""
library("gcmr")
data("epilepsy", package = "gcmr")
"""
@rget epilepsy

# form model
df = epilepsy
y = :counts
grouping = :id
d = NegativeBinomial() # Normal() # Poisson()
link = LogLink() # IdentityLink() # LogLink()
covariates = [:visit, :trt]
qc_model = AR_model(df, y, grouping, covariates, d, link; penalized=true)

# fit
QuasiCopula.fit!(qc_model)

Initializing NegBin r to Poisson regression values
initializing β using Newton's Algorithm under Independence Assumption
initializing variance components using MM-Algorithm
initializing r using Newton update
Converging when tol ≤ 1.0e-6 (max block iter = 10)
Block iter 1 r = 1.31, logl = -990.38, tol = 0.6098788177411643
Block iter 2 r = 1.3, logl = -989.53, tol = 0.0008615566863290024
Block iter 3 r = 1.3, logl = -989.52, tol = 7.787968897507412e-6


-989.5192980240267

In [13]:
@show qc_model.β
@show qc_model.σ2
@show qc_model.τ
@show qc_model.ρ;

qc_model.β = [3.4708028031628664, -1.3680429994395282, -0.052249866925391415]
qc_model.σ2 = [0.3167502942469257]
qc_model.τ = [1.0]
qc_model.ρ = [1.0000000099989756]


In [14]:
qc_model.r # negative binomial only

1-element Vector{Float64}:
 1.2998208401866174

# CS

In [1]:
using Revise
using Ipopt
using QuasiCopula
using Random
using GLM
using LinearAlgebra
using Statistics
using StatsBase
using MathOptInterface
using RCall

Precompiling QuasiCopula
  ✓ QuasiCopula
  1 dependency successfully precompiled in 5 seconds. 151 already precompiled.


In [21]:
# get data
R"""
data(respiratory, package="geepack")
respiratory_df <- respiratory[order(respiratory$id),]
"""
@rget respiratory_df

# form model
df = respiratory_df
y = :outcome
grouping = :id
d = NegativeBinomial() # Normal() # Bernoulli()
link = LogLink() # IdentityLink() # LogitLink()
covariates = [:center, :age, :baseline]
qc_model = CS_model(df, y, grouping, covariates, d, link)

# fit
QuasiCopula.fit!(qc_model)

Initializing NegBin r to Poisson regression values
initializing β using Newton's Algorithm under Independence Assumption
initializing r using Newton update
Converging when tol ≤ 1.0e-6 (max block iter = 10)
Block iter 1 r = 3998.98, logl = -378.93, tol = 0.06387019167253356
Block iter 2 r = 230618.94, logl = -377.78, tol = 0.003040588283321018
Block iter 3 r = 1.515384126e7, logl = -377.75, tol = 6.562430030814395e-5
Block iter 4 r = 9.51439913e6, logl = -377.75, tol = 7.502032912530231e-7


-377.7509205824111

In [22]:
@show qc_model.β
@show qc_model.σ2
@show qc_model.ρ;

qc_model.β = [-1.8401217795320461, 0.42653463322121343, -0.006493382719069296, 0.6399440247754395]
qc_model.σ2 = [0.5977390678291081]
qc_model.ρ = [1.0000000099910034]
